<a href="https://www.kaggle.com/code/nishithanishitha/fake-news-detection-lr?scriptVersionId=234947838" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import string
import nltk

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fake-news-detection-datasets/News _dataset/True.csv
/kaggle/input/fake-news-detection-datasets/News _dataset/Fake.csv


In [2]:
data_True = pd.read_csv("/kaggle/input/fake-news-detection-datasets/News _dataset/True.csv")
data_Fake = pd.read_csv("/kaggle/input/fake-news-detection-datasets/News _dataset/Fake.csv")

In [3]:
data_True.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [4]:
data_Fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [5]:
#lets check if data is null 
print("For True")
data_True.isnull().sum()

For True


title      0
text       0
subject    0
date       0
dtype: int64

In [6]:
print("For Fake")
data_Fake.isnull().sum()

For Fake


title      0
text       0
subject    0
date       0
dtype: int64

label the data

In [7]:
data_True['label'] = 1

In [8]:
data_Fake['label'] = 0

In [9]:
data_True.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [10]:
data_Fake.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [11]:
data=pd.concat([data_True, data_Fake], ignore_index=True, axis=0)

In [12]:
data = data.sample(frac=1, random_state=42).reset_index(drop=True)
data.head()

,title,text,subject,date,label
0,BREAKING: GOP Chairman Grassley Has Had Enoug...,"Donald Trump s White House is in chaos, and th...",News,"July 21, 2017",0
1,Failed GOP Candidates Remembered In Hilarious...,Now that Donald Trump is the presumptive GOP n...,News,"May 7, 2016",0
2,Mike Pence’s New DC Neighbors Are HILARIOUSLY...,Mike Pence is a huge homophobe. He supports ex...,News,"December 3, 2016",0
3,California AG pledges to defend birth control ...,SAN FRANCISCO (Reuters) - California Attorney ...,politicsNews,"October 6, 2017",1
4,AZ RANCHERS Living On US-Mexico Border Destroy...,Twisted reasoning is all that comes from Pelos...,politics,"Apr 25, 2017",0


In [13]:
data['Combined'] = data['title'].astype(str) + data['text']

In [14]:
newdata = data.drop(['title','text','subject','date'],axis='columns')
newdata.head()

,label,Combined
0,0,BREAKING: GOP Chairman Grassley Has Had Enoug...
1,0,Failed GOP Candidates Remembered In Hilarious...
2,0,Mike Pence’s New DC Neighbors Are HILARIOUSLY...
3,1,California AG pledges to defend birth control ...
4,0,AZ RANCHERS Living On US-Mexico Border Destroy...


In [15]:
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def wordopt(text):
    # Lowercase the text
    text = text.lower()

    # Remove unwanted characters and patterns
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub("\\W", " ", text)

    # Tokenize and remove stopwords
    word_list = text.split()
    text_clean = [word for word in word_list if word not in stop_words]

    # Join back to string
    return ' '.join(text_clean)


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
newdata['Combined'] = newdata['Combined'].apply(wordopt)

In [17]:
newdata.head()

,label,Combined
0,0,breaking gop chairman grassley enough demands ...
1,0,failed gop candidates remembered hilarious moc...
2,0,mike pence new dc neighbors hilariously trolli...
3,1,california ag pledges defend birth control ins...
4,0,az ranchers living usmexico border destroy nan...


In [18]:
#split the data 
x = data['Combined']
y = data['label']

In [19]:
# !pip install scikit-learn

In [20]:
# Splitting the dataset into training set and testing set.

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.25)

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [22]:
from sklearn.linear_model import LogisticRegression

In [23]:
model = LogisticRegression()
model.fit(xv_train,y_train)

LogisticRegression()

In [24]:
pred_lr = model.predict(xv_test)

model.score(xv_test, y_test)

0.9860133630289533

In [25]:
from sklearn.metrics import classification_report

print (classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5885
           1       0.99      0.99      0.99      5340

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [26]:
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"

def testing(news):
    testing_news = {"text": [news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test['text'] = new_def_test["text"].apply(wordopt)
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)

    pred_LR = model.predict(new_xv_test)
    print("\nLR Prediction:", output_lable(pred_LR[0]))


In [27]:
#testing for fake news
#for user input
# news=str(input())
# testing(news)
# testing for fake news
news = " The rumour began after an X user shared a quote attributed to Zinta, supposedly in response to Pant’s earlier remark that he was only concerned about not being picked by Punjab Kings during the auction. “We had both Rishabh Pant and Shreyas Iyer – options we could have taken… but we wanted a big performer,” the user quoted Zinta as saying."
testing(news)



LR Prediction: Fake News


In [28]:
#testing for real news
# news=str(input())
# testing(news)
news = '''According to the Kremlin, the ceasefire will last from 6 p.m. Moscow time (3 p.m. GMT) on Saturday (April 19, 2025) to midnight (9 p.m. GMT) following Easter Sunday (April 20, 2025).

“Guided by humanitarian considerations, today from 18:00 (6 p.m.) 00:00 (12 a.m.) from Sunday to Monday, the Russian side declares an Easter truce. I order that all military actions be stopped for this period,” Mr. Putin said at a meeting with Chief of the General Staff Valery Gerasimov, the Kremlin’s Press Service quoted him as saying.

“We assume that the Ukrainian side will follow our example. At the same time, our troops must be ready to repel possible violations of the truce and provocations from the enemy, any of its aggressive actions,” Mr. Putin said.

Mr. Putin’s announcement came after U.S. President Donald Trump on Friday (April 18, 2025) said negotiations between Ukraine and Russia are “coming to a head” and insisted that neither side is “playing” him in his push to end the grinding three-year war.

Mr. Trump spoke shortly after Secretary of State Marco Rubio warned that the U.S. may “move on” from trying to secure a Russia-Ukraine peace deal if there is no progress in the coming days, after months of efforts have failed to bring an end to the fighting.'''
testing(news)


LR Prediction: Not A Fake News
